# Clean Stanford Meter Data
Code author: Sahar H. El Abbadi
Date started: 2022-03-11
Date last edited: 2022-03-11

## Data description:
Stanford metering data, cleaned according to data processing pipeline. Includes 30-s, 60-s, and 90-s average flow rates, percent methane, and whole gas measurements. Each measurement is provided for the three different timestamp metrics: Stanford timestamps, as observed from the field; operator reported timestamps; and FlightRadar timestamps.

## Script objective:
Select data for the desired timestamp and time average. Make column names machine-readable.

In [1]:
# Setup

# Imports
import pandas as pd
import pathlib
from methods_source import make_operator_meter_dataset

# Load data

# Carbon Mapper meter data
cm_path = pathlib.PurePath('02_meter_data', 'summary_files', 'CM.csv')
cm_meter_raw = pd.read_csv(cm_path)

# GHGSat meter data
ghg_path = pathlib.PurePath('02_meter_data', 'summary_files', 'GHGSat.csv')
ghg_meter_raw = pd.read_csv(ghg_path)

# Kairos meter data
kairos_path = pathlib.PurePath('02_meter_data', 'summary_files', 'Kairos.csv')
kairos_meter_raw = pd.read_csv(kairos_path)

# MAIR meter data
mair_path = pathlib.PurePath('02_meter_data', 'summary_files', 'MAIR.csv')
mair_meter_raw = pd.read_csv(mair_path)

ModuleNotFoundError: No module named 'methods_analysis'

In [ ]:

# Cannot currently run Methane Air team, so removing it from the for loop for now

timekeepers = ['flightradar', 'stanford', 'team']
operators = ['Carbon Mapper', 'GHGSat', 'Kairos']

# Dictionary of raw meter data
meter_raw = {
    'Carbon Mapper': cm_meter_raw,
    'GHGSat': ghg_meter_raw,
    'Kairos': kairos_meter_raw,
}

# Generate all datasets and save them
for timekeeper in timekeepers:
    for operator in operators:
        make_operator_meter_dataset(operator, meter_raw[operator], timekeeper)


cm_meter = make_operator_meter_dataset('Carbon Mapper', cm_meter_raw, 'flightradar')
ghg_meter = make_operator_meter_dataset('GHGSat', ghg_meter_raw, 'flightradar')
kairos_meter = make_operator_meter_dataset('Kairos', kairos_meter_raw, 'flightradar')
# mair_meter = make_operator_meter_dataset('Methane Air', mair_meter_raw, 'flightradar')

